In [ ]:
from pathlib import Path
import datetime

import matplotlib.pyplot as plt
import polars as pl

import data_fetcher

In [ ]:
start = datetime.datetime(2024, 12, 3)
end = datetime.datetime(2024, 12, 5)

In [ ]:
fx_fetcher = data_fetcher.histdata.HistDataFetcher()
fx_df = fx_fetcher.fetch_ticker("usdjpy", start_date=start - datetime.timedelta(seconds=10), end_date=end + datetime.timedelta(seconds=10))

bin_fetcher = data_fetcher.binance.BinanceFetcher()
bin_df = bin_fetcher.fetch_ticker("BTCUSDT", start_date=start, end_date=end)

In [ ]:
gmo_fetcher = data_fetcher.gmo.GMOFethcer()
gmo_df = gmo_fetcher.fetch_ticker("BTC_JPY", start_date=start, end_date=end)

In [ ]:
res = fx_df["datetime"].search_sorted(bin_df["datetime"])
bid, ask = fx_df["bid"][res], fx_df["ask"][res]

In [ ]:
bin_df = bin_df.with_columns(
    bid.alias("usdjpy_bid"),
    ask.alias("usdjpy_ask"),
).with_columns(
    (pl.col("price") * pl.col("usdjpy_bid")).alias("price_jpy_bid"),
    (pl.col("price") * pl.col("usdjpy_ask")).alias("price_jpy_ask"),
)

In [ ]:
bin_df

In [ ]:
st = start + datetime.timedelta(minutes=0)
et = st + datetime.timedelta(minutes=1)

bdf = bin_df.filter(pl.col("datetime").is_between(st, et))
gdf = gmo_df.filter(pl.col("datetime").is_between(st, et))

#plt.plot(bdf["datetime"], bdf["price_jpy_bid"] / bdf["price_jpy_bid"][0], label="bid")
# plt.plot(bdf["datetime"], bdf["price"] / bdf["price"][0], label="bin")
plt.plot(bdf["datetime"], bdf["price_jpy_ask"] / bdf["price_jpy_ask"][0], label="ask")
plt.plot(gdf["datetime"], gdf["price"] / gdf["price"][0], label="gmo")
plt.legend()
plt.grid()

In [ ]:
gmo_df